Операторы **in** и **not in** используют "магический" метод \_\_contains\_\_, который возвращает **True**, если переданный элемент содержиться в экземпляре класса.

По умолчанию метод \_\_contains\_\_ реализован через протокол итераторов:

In [1]:
class object:
    # ...
    
    def __contains__(self, target):
        for item in self:
            if item == target:
                return True
            return False

Протокол итераторов состоит из двух обязательных методов

+ Метод \_\_iter\_\_ возвращает экземпляр класса, реализующего протокол итераторов, например **self**
+ Метод \_\_next\_\_ возвращает следующий по порядку элемент итератора. Если такого элемента нет, то метод должен подянть исключение **StopIteration**

В Python реализован упрощенный вариантреализации протокола итераторов с использованием метода \_\_getitem\_\_.

Метод \_\_getitem\_\_ принимает один аргумент -  индекс элемента в последовательности и

+ либо возвращает элемент, соответствующий индексуб
+ либо поднимает **IndexError**, если элемента с таким индексом нет

In [2]:
class Identity:
    def __getitem__(self, idx):
        if idx > 5:
            raise IndexError(idx)
        else:
            return idx        

In [3]:
list(Identity())

[0, 1, 2, 3, 4, 5]

In [4]:
# "Семантика" упрощенного протокола итераторов : seq_iter
class seq_iter:
    def __init__(self, instance):
        self.instance = instance
        self.idx = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        try:
            res = self.instance[self.idx]
        except IndexError:
            raise StopIteration
            
        self.idx += 1
        return res

In [5]:
# Семантика упрощенного протокола итераторов : object

class object:
    # ...
    
    def __iter__(self):
        if not hasattr(self, "__getitem__"):
            cls = self.__class__
            msg = "{} object is not iterable"
            raise TypeError(msg.format(cls.__name__))
        return seq_iter(self)

In [6]:
# Известное поведение, с которым ничего нельзя сделать это исчерпывание итератора
it = iter([1, 2, 3])
for x in it: pass
print(list(it))

[]


Любой итератор это iterable, a iterable это не итератор, потому что у него нет \_\_next\_\_

Резюме

В Python iteartor также явялется iterable.

Итератор - это экземпляр класса, который реализует два метода:
+ \_\_iter__ 
+ \_\_next__

Альтернативно можно воспользоваться реалищацией этих методов по умолчанию и определить метод \_\_getitem__  

Протокол итераторов используется:

+ оператором **for**
+ операторами **in** и **not in**

Протокол итераторов реализуется всеми встроенными колекциями, а также файлами и объектами типа map, filter и т.д.

### Генераторы

Генератор - это функция, которая использует не только оператор **return** но и оператор **yield**

В результате работы оператора **yield** работа функции приостанавливается, а не прерывается как в случае оператора **return**. 

In [3]:
# Пример
def g():
    print("Started")
    x = 42
    yield x
    x += 1
    yield x
    print("Done")

In [4]:
type(g)

function

In [5]:
gen = g()

In [6]:
type(gen)

generator

In [7]:
next(gen)

Started


42

In [8]:
next(gen)

43

In [10]:
next(gen)

Done


StopIteration: 

In [11]:
g()

<generator object g at 0x7fed07a56750>

In [8]:
# Пример генератора на примере создания итератора, функции, возвращающей уникальные элементы
def unique(iterable, seen=None):
    seen = set(seen or [])
    for item in iterable:
        if item not in seen:
            seen.add(item)
            yield item

In [9]:
xs = [1,1,4,2,3]

In [10]:
unique(xs)

<generator object unique at 0x0000025765451888>

In [11]:
list(unique(xs))

[1, 4, 2, 3]

In [12]:
1 in unique(xs)

True

In [13]:
# Пример генератора на примере реализации встроенной функции map

def mymap(func, iterable, *rest):
    for args in zip(iterable, *rest):
        yield func(*args)

In [14]:
xs = range(5)

In [15]:
mymap(lambda x : x * x, xs)

<generator object mymap at 0x0000025765451D00>

In [16]:
list(mymap(lambda x : x * x, xs))

[0, 1, 4, 9, 16]

In [17]:
9 in mymap(lambda x : x * x, xs)

True

Переиспользование генераторов не допускается

In [18]:
def g():
    yield 42

In [19]:
gen = g()

In [20]:
list(gen)

[42]

In [21]:
list(gen)

[]

В Python есть генераторы списков, множеств и словарей

Выражения-генераторы работают аналогичным образом, но не порождают коллекцию в процессе работы:

In [22]:
gen = (x ** 2 for x in range(10**42) if x % 2 == 1) # по хорошему, такая коллекция не влезет в память

In [23]:
gen

<generator object <genexpr> at 0x0000025765451E08>

In [24]:
next(gen)

1

In [25]:
# Если выражение-генератор - единственный аргумент функции, скобки можно опустить:
sum(x ** 2 for x in range(10) if x % 2 == 1)

165

Выражение yield

Оператор yield может быть использован как выражение

In [26]:
def g():
    res = yield # точка входа 1
    print("Got {!r}".format(res))
    res = yield 42 # точка входа 2
    print("Got {!r}".format(res))

In [27]:
gen = g()

In [28]:
next(gen) # Промотаем до первого yield

In [29]:
next(gen) # Промотаем до второго yield

Got None


42

In [30]:
next(gen) # выполним оставшуюся часть генератора

Got None


StopIteration: 

Может показаться, что выражение **yield** выглядит бесполезно, но

Существует метод **send** который **возобновляет** выполнение генератора и "отправляет" свой аргумент в следующий yield

In [31]:
gen = g()

In [32]:
gen.send('foobar')

TypeError: can't send non-None value to a just-started generator

Чтобы инициализировать генератор нужно "отправить" ему **None**. Функция **next** делает ровно это:

In [33]:
gen = g()
next(gen)

Результатом метода **send** является следующее значение генератора или исключение **StopIteration**, если такого исключения нет.

In [34]:
gen = g()
gen.send(None) # ~ next(gen) "иничиализация генератора"
gen.send("foobar")

Got 'foobar'


42

Помимо этого существует метод **throw** поднимающий переданное исключение в месте, где генератор приостоновил исполнение и возвращает следующее значение генератора:

In [35]:
def g():
    try:
        yield 42
    except Exception as e:
        yield e

In [36]:
gen = g()

In [37]:
next(gen)

42

In [38]:
gen.throw(ValueError, "something is wrong")

ValueError('something is wrong')

In [39]:
gen.throw(RuntimeError, "another error")

RuntimeError: another error

Если генератор не обработал брошенное в него исключение, то выполнение генератора прекращается и исключение подается наверх по стеку вызовов.

Имеющийся в арсенала генераторов метод **close** поднимает специальное исключение **GeneratorExit** в месте, где генератор приостоновил исполнение:

In [40]:
def g():
    try:
        yield 42
    finally:
        print("done")

In [41]:
gen = g()

In [42]:
next(gen)

42

In [43]:
gen.close()

done


Если все хорошо, то метод **close** завершит работу генератора и ничего не вернет.

Генератор может обработать исключение **GeneratorExit** и поднять другое исключение.

#### Генераторы ~ сопрограммы aka coroutines 
*(http://dabeaz.com/coroutines)*

Сопрограммы - это программы, которые могут иметь больше одной точки входа, а также поддерживать остановку и продолжение с сохранением состояния

In [44]:
def grep(pattern):
    print("Looking for {!r}".format(pattern))
    while True:
        line = yield
        if pattern in line:
            print(line)

In [45]:
gen = grep("Gotcha!")

In [46]:
next(gen)

Looking for 'Gotcha!'


In [47]:
gen.send("This line doesn't have what we're lookong for")

In [48]:
gen.send("This one does. Gotcha!")

This one does. Gotcha!


Прежде чем работать с сопрограммой ее нужно инициализировать с помощью вызова функции **next**

Объявим декоратор **coroutine** который скроет эту деталь реализации:


In [53]:
def coroutine(g):
    @functools.wraps(g)
    def inner(*args, **kwargs):
        gen = g(*args, **kwargs)
        next(gen)
        return gen
    return inner

In [54]:
grep = coroutine(grep)
gen = grep("Gotcha!")
gen.send("One more line for ya! Gotcha!")

NameError: name 'functools' is not defined

#### Оператор yield from

Оператор yield from позволяет делегировать выполнение другому генератору:

In [55]:
def chain(*iterables):
    for iterable in iterables:
        yield from iterable

Любые вызовы методом **send** и **throw** у родительсокго генератора будет переданы вложенному генератору без изменений.

#### Замена менеджера контекста на генератор

+ Протокол менеджеров контекста требует реализации двух методов:
    + \_\_enter__
    + \_\_exit__
+ Если мы хотим, что бы у менеджера было какое-то состояние, то мы вынуждены добавить метод \_\_init__

In [56]:
class cd:
    def __init__(self, path):
        self.path = path
    
    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)
        
    def __exit__(self, *exc_info):
        os.chdir(self, saved_cwd)

Декоратор contextmanager из модуля contextlib принимает генератор специального вида и строит по нему менеджер контекста.

In [57]:
import os
from contextlib import contextmanager

In [58]:
@contextmanager
def cd(path):               # __init__
    old_path = os.getcwd()  # __enter__
    os.chdir(path)
    try:
        yield               # __________
    finally:
        os.chdir(old_path)  # __exit__

Генераторы позволяют сократить количество синтаксического шума при реализации менеджеров контекста.

Еще один премер, создание временной директории

In [60]:
import tempfile
import shutil

@contextmanager
def tempdir():                  # __init__
    outdir = tempfile.mkdtemp() # __enter__
    try:
        yield outdir            # __________ 
    finally:
        shutil.rmtree(outdir)   # __exit__

In [61]:
with tempdir() as path:
    print(path)

C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp1cwk0iew


### Модуль itertools

#### Пример: функция islice

Функция islice обобщает понятие слайса на произвольный итератор:

In [62]:
from itertools import islice

In [63]:
xs = range(10)

In [79]:
list(islice(xs, 3))

[0, 1, 2]

In [65]:
list(islice(xs, 3, None))

[3, 4, 5, 6, 7, 8, 9]

In [66]:
list(islice(xs, 3, 8, 2))

[3, 5, 7]

#### Пример: бесконечные итераторы

In [76]:
"""
Для удобства реализуем родственника функции drop, функцию take, которая строит список из более чем
n первых элементов, переданного ей итератора
"""
def take(n, iterable):
    return list(islice(iterable, n))

In [81]:
take(2, range(10))

[0, 1]

In [82]:
from itertools import count, cycle, repeat

In [84]:
take(4, count(0, 5))

[0, 5, 10, 15]

In [85]:
take(5, cycle([1, 2, 3]))

[1, 2, 3, 1, 2]

In [86]:
take(5, repeat(42))

[42, 42, 42, 42, 42]

In [87]:
take(5, repeat(42, 2))

[42, 42]